### 텐서 다루기

In [4]:
"""
import torch
print(torch.tensor([[1,2],[3,4]]))
print(torch.tensor([[1,2],[3,4]], device='cuda:0'))
print(torch.tensor([[1,2],[3,4]], device='cuda:0', dtype=torch.float64))
"""
import torch

# 기본 CPU 텐서
print(torch.tensor([[1, 2], [3, 4]]))

# MPS(Apple Metal) 디바이스 텐서
print(torch.tensor([[1, 2], [3, 4]], device='mps'))

# MPS 디바이스 텐서 + float32 dtype
print(torch.tensor([[1, 2], [3, 4]], device='mps', dtype=torch.float32))


tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]], device='mps:0')
tensor([[1., 2.],
        [3., 4.]], device='mps:0')


In [2]:
import torch

print("PyTorch 버전:", torch.__version__)
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("GPU 개수:", torch.cuda.device_count())
print("현재 사용 중인 GPU:", torch.cuda.current_device() if torch.cuda.is_available() else "GPU 사용 불가")
print("GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "GPU 사용 불가")


PyTorch 버전: 2.5.1
CUDA 사용 가능 여부: False
GPU 개수: 0
현재 사용 중인 GPU: GPU 사용 불가
GPU 이름: GPU 사용 불가


In [8]:
temp=torch.tensor([[1,2],[3,4]])
print(temp.numpy()) 
temp=torch.tensor([[1,2],[3,4]], device='mps')
print(temp.to('cpu').numpy())

[[1 2]
 [3 4]]
[[1 2]
 [3 4]]


In [11]:
temp=torch.FloatTensor([1,2,3,4,5,6,7])
print(temp[0],temp[1],temp[-1])
print('------------------------')
print(temp[2:5],temp[4:-1])

tensor(1.) tensor(2.) tensor(7.)
------------------------
tensor([3., 4., 5.]) tensor([5., 6.])


In [12]:
v=torch.tensor([1,2,3])
w=torch.tensor([3,4,6])
print(w-v)

tensor([2, 2, 3])


In [13]:
#텐서의 차원을 조작
temp=torch.tensor([[1,2],[3,4]])

print(temp.shape)
print('------------------------')
print(temp.view(4,1))
print('------------------------')
print(temp.view(-1))
print('------------------------')
print(temp.view(1,-1))  #-1은 (1,?)과 같은 의미
print('------------------------')
print(temp.view(-1,1))


torch.Size([2, 2])
------------------------
tensor([[1],
        [2],
        [3],
        [4]])
------------------------
tensor([1, 2, 3, 4])
------------------------
tensor([[1, 2, 3, 4]])
------------------------
tensor([[1],
        [2],
        [3],
        [4]])


### 데이터 준비

In [ ]:
import pandas as pd
import torch 
data=pd.read_csv('../class2.csv')

x=torch.from_numpy(data['x'].values).unsqueeze(dim=1).float()
y=torch.from_numpy(data['y'].values).unsqueeze(dim=1).float()

custom dataset : 데이터를 한번에 다 부르지 않고 조금씩 나누어 불러서 사용하는 방식

In [ ]:
class customDataset(torch.utils.data.Dataset):
    def __init__(self):
    def __len__(self):
    def __getitem__(self,index):

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.label=pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self,idx):
        sample=torch.tensor(self.label.iloc[idx,0:3]).int()
        label=torch.tensor(self.label.iloc[idx,3]).int()
        return sample,label
tensor_dataset=CustomDataset('../covtype.csv')
tensor_loader=DataLoader(tensor_dataset,batch_size=4,shuffle=True)

In [23]:
import torchvision.transforms as transforms
mnist_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(1.0,))
])
from torchvision.datasets import MNIST
import requests
download_root='../chap02/data/MNIST_DATASET'
train_dataset=MNIST(download_root,train=True,transform=mnist_transform,download=True)
valid_dataset=MNIST(download_root,train=False,transform=mnist_transform,download=True)
test_dataset=MNIST(download_root,train=False,transform=mnist_transform,download=True)

### 모델 정의

In [ ]:
model=nn.Linear(in_features=1, out_features=1, bias=True)

In [27]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))
        
        self.layer2=nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))
        
        self.layer3=nn.Sequential(
            nn.Linear(in_features=30*5*5, out_features=10, bias=True),
            nn.ReLU(inplace=True))
        
        def forward(self,x):
            x=self.layer1(x)
            x=self.layer2(x)
            x=x.view(x.size(0),-1)
            x=self.layer3(x)
            return x
model=MLP()

print('Printing children\n-------------------')
print(list(model.children()))
print('\n\nPrinting modules\n-------------------')
print(list(model.modules()))
        

Printing children
-------------------
[Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Linear(in_features=750, out_features=10, bias=True)
  (1): ReLU(inplace=True)
)]


Printing modules
-------------------
[MLP(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=750, out

In [28]:
def MLP(in_features=1, hidden_features=1, out_features=1):
    hidden=nn.Linear(in_features, out_features=hidden_features, bias=True)
    activation=nn.ReLU()
    output=nn.Linear(hidden_features, out_features, bias=True)
    net=nn.Sequential(hidden,activation,output)
    return net

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.95**epoch)

for epoch in range(1, 100+1):
    for x, y in DataLoader:
        optimizer.zero_grad()
        loss = criterion(model(x), y)  
        loss.backward()
        optimizer.step()
    scheduler.step()



### 모델 훈련

In [ ]:
for epoch in range(100):
    yhat=model(x_train)
    loss=criterion(yhat,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### 모델 평가

In [ ]:
import torch 
import torchmetrics
preds=torch.randn(10,5).softmax(dim=1)
target=torch.randint(0,5,(10,))
acc=torchmetrics.functional.accuracy(preds,target)

In [ ]:
import torch
import torchmetrics
metric=torchmetrics.Accuracy()

n_batches=10
for i in range(n_batches):
    preds=torch.randn(10,5).softmax(dim=-1)
    target=torch.randint(5,(10,))
    
    acc=metric(preds,target)
    print(f"Accuray on batch {i}: {acc}")
acc=metric.compute()
print(f"Final accuracy: {acc}")

### 훈련 과정 모니터링

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer=SummaryWriter("../chap02/tensorboard")

for epoch in range(num_epochs):
    model.train()
    batch_loss=0.0
    
    for i, (x,y) in enumerate(dataloader):
        x,y=x.to(device).float(),y.to(device).float()
        outputs=model(x)
        loss=criterion(outputs,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
writer.close()

#### model.train() & model.eval()

In [ ]:
model.eval()
with torch.no_grad():
    valid_loss=0
    
    for x,y in valid_dataloader:
        outputs=model(x)
        loss=F.cross_entropy(outputs,y.long().squeeze())
        valid_loss+=float(loss)
        y_hat+=[outputs]
        
valid_loss=valid_loss/len(valid_dataloader)

### 파이토치 코드 맛보기

In [53]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

dataset=pd.read_csv('car_evaluation.csv')
dataset.head()

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [56]:
fig_size=plt.rcParams['figure.figsize']
fig_size[0]=8
fig_size[1]=6
plt.rcParams['figure.figsize']=fig_size
dataset.output.value_counts().plot(kind='pie',autopct='%0.05f%%',
                                    colors=['lightblue','lightgreen','orange','pink'],explode=(0.05,0.05,0.05,0.05))

<Axes: ylabel='count'>

In [57]:
#예제 데이터셋 칼럼들의 목록
categorial_columns=['price','maint','doors','persons','lug_capacity','safety']

#astype() 메서드를 이ㅛㅇ하여 데이터를 범주형으로 변환
for category in categorial_columns:
    dataset[category]=dataset[category].astype('category')
    
price=dataset['price'].cat.codes.values
maint=dataset['maint'].cat.codes.values
doors=dataset['doors'].cat.codes.values

persons=dataset['persons'].cat.codes.values
lug_capacity=dataset['lug_capacity'].cat.codes.values
safety=dataset['safety'].cat.codes.values

catagorical_data=np.stack([price,maint,doors,persons,lug_capacity,safety],1)
catagorical_data[:10]

array([[3, 3, 0, 0, 2, 1],
       [3, 3, 0, 0, 2, 2],
       [3, 3, 0, 0, 2, 0],
       [3, 3, 0, 0, 1, 1],
       [3, 3, 0, 0, 1, 2],
       [3, 3, 0, 0, 1, 0],
       [3, 3, 0, 0, 0, 1],
       [3, 3, 0, 0, 0, 2],
       [3, 3, 0, 0, 0, 0],
       [3, 3, 0, 1, 2, 1]], dtype=int8)

In [58]:
outputs=pd.get_dummies(dataset.output)
outputs=outputs.values
outputs=torch.tensor(outputs).flatten()

print(catagorical_data.shape)
print(outputs.shape)

(1728, 6)
torch.Size([6912])


In [59]:
import pandas as pd
import numpy as np

data={
    'gender':['male','female','male'],
    'weight':[72,55,58],
    'nation':['Japan','Korea','Australia']
}

df=pd.DataFrame(data)
df

,gender,weight,nation
0,male,72,Japan
1,female,55,Korea
2,male,58,Australia


In [60]:
pd.get_dummies(df)

,weight,gender_female,gender_male,nation_Australia,nation_Japan,nation_Korea
0,72,False,True,False,True,False
1,55,True,False,False,False,True
2,58,False,True,True,False,False


In [62]:
a=np.array([[1,2],[3,4]])
print(a.ravel())
print(a.reshape(-1))
print(a.flatten())

[1 2 3 4]
[1 2 3 4]
[1 2 3 4]


In [63]:
categorical_column_sizes=[len(dataset[column].cat.categories) for column in categorial_columns]
categorical_embedding_sizes=[(col_size,min(50,(col_size+1)//2)) for col_size in categorical_column_sizes]
print(categorical_embedding_sizes)

[(4, 2), (4, 2), (4, 2), (3, 2), (3, 2), (3, 2)]


In [64]:
total_records=1728
test_records=int(total_records*0.2)

catagorical_train_data=catagorical_data[:total_records-test_records]
catagorical_test_data=catagorical_data[total_records-test_records:total_records]
train_outputs=outputs[:total_records-test_records]
test_outputs=outputs[total_records-test_records:total_records]

In [65]:
print(len(catagorical_train_data))
print(len(train_outputs))
print(len(catagorical_test_data))
print(len(test_outputs))

1383
1383
345
345


In [67]:
class Model(nn.Module):
    def __init__(self,embedding_size,output_size, layers, p=0.4):
        super().__init__()
        self.all_embeddings=nn.ModuleList([nn.Embedding(ni,nf) for ni,nf in embedding_size])
        self.embedding_dropout=nn.Dropout(p)
        
        all_layers=[]
        num_categorical_cols=sum((nf for ni,nf in embedding_size))
        input_size=num_categorical_cols
        for i in layers:
            all_layers.append(nn.Linear(input_size,i))
            all_layers.append(nn.ReLU(inplace=True))
            all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            input_size=i
            
        all_layers.append(nn.Linear(layers[-1],output_size))
        self.layers=nn.Sequential(*all_layers)
        
    def forward(self,x_categorical):
        embeddings=[]
        for i,e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:,i]))
        x=torch.cat(embeddings,1)
        x=self.embedding_dropout(x)
        x=self.layers(x)
        return x

In [68]:
model=Model(categorical_embedding_sizes,4,[200,100,50],p=0.4)
print(model)

Model(
  (all_embeddings): ModuleList(
    (0-2): 3 x Embedding(4, 2)
    (3-5): 3 x Embedding(3, 2)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_features=4, bias=True)
  )
)


In [69]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [71]:
device=torch.device('mps')

In [74]:
import torch

# 예: device 설정 (Apple Silicon(M1/M2/M3) 환경이면 "mps", 아니면 "cuda" 또는 "cpu")
device = "mps" if torch.backends.mps.is_available() else "cpu"

# 모델을 device로 이동
model = model.to(device)

# 데이터를 텐서로 변환 후 device에 올림 (dtype은 임베딩 등에서 long 필요)
catagorical_train_data = torch.tensor(catagorical_train_data, dtype=torch.long, device=device)
train_outputs = torch.tensor(train_outputs, dtype=torch.long, device=device)

epochs = 500
aggregated_losses = []

for epoch in range(1, epochs + 1):
    # 순전파(Forward)
    y_pred = model(catagorical_train_data)
    single_loss = loss_function(y_pred, train_outputs)
    
    # 손실값 저장 (item()으로 파이썬 float 추출)
    aggregated_losses.append(single_loss.item())
    
    # 일정 간격으로 학습 상황 출력
    if epoch % 25 == 1:
        print(f"epoch: {epoch:3d} loss: {single_loss.item():10.8f}")
    
    # 역전파(Backward) 및 최적화(Optimization)
    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

# 최종 에폭 결과 출력
print(f"epoch: {epoch:3d} loss: {single_loss.item():10.10f}")


/var/folders/7p/npc42ml14hsdz2qfn43lc51m0000gn/T/ipykernel_74464/2731399717.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  catagorical_train_data = torch.tensor(catagorical_train_data, dtype=torch.long, device=device)
/var/folders/7p/npc42ml14hsdz2qfn43lc51m0000gn/T/ipykernel_74464/2731399717.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_outputs = torch.tensor(train_outputs, dtype=torch.long, device=device)


epoch:   1 loss: 1.60019720
epoch:  26 loss: 1.41587591
epoch:  51 loss: 1.30578458
epoch:  76 loss: 1.19392371
epoch: 101 loss: 1.08023822
epoch: 126 loss: 0.93049651
epoch: 151 loss: 0.82374090
epoch: 176 loss: 0.73694879
epoch: 201 loss: 0.70405132
epoch: 226 loss: 0.66560006
epoch: 251 loss: 0.64449430
epoch: 276 loss: 0.62348288
epoch: 301 loss: 0.60305870
epoch: 326 loss: 0.60607475
epoch: 351 loss: 0.59729302
epoch: 376 loss: 0.59649140
epoch: 401 loss: 0.58034122
epoch: 426 loss: 0.57848531
epoch: 451 loss: 0.57330346
epoch: 476 loss: 0.58233160
epoch: 500 loss: 0.5781533122


In [76]:
# catagorical_test_data를 LongTensor로 변환
catagorical_test_data = torch.tensor(catagorical_test_data, dtype=torch.long, device=device)

test_outputs = test_outputs.to(device=device, dtype=torch.int64)

with torch.no_grad():
    y_val = model(catagorical_test_data)  # 이제 오류 없이 텐서로 입력
    loss = loss_function(y_val, test_outputs)
print(f'Loss: {loss:.8f}')


Loss: 0.56123626


In [77]:
print(y_val[:5])

tensor([[ 3.3719,  2.1440, -3.7218, -3.7182],
        [ 1.7963,  1.0036, -2.3565, -2.1838],
        [ 2.3649,  1.5639, -3.3986, -3.6514],
        [ 2.7980,  1.4888, -3.0753, -3.3044],
        [ 1.6430,  0.9629, -2.0581, -2.2644]], device='mps:0')


In [79]:
# y_val이 mps 텐서라면:
y_val_argmax = y_val.argmax(dim=1)  # axis=1에 해당
print(y_val_argmax[:5])


tensor([0, 0, 0, 0, 0], device='mps:0')


In [83]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
print(confusion_matrix(test_outputs.cpu(),y_val_argmax.cpu()))
print(classification_report(test_outputs.cpu(),y_val_argmax.cpu()))
print(accuracy_score(test_outputs.cpu(),y_val_argmax.cpu()))

[[259   0]
 [ 86   0]]
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       259
           1       0.00      0.00      0.00        86

    accuracy                           0.75       345
   macro avg       0.38      0.50      0.43       345
weighted avg       0.56      0.75      0.64       345

0.7507246376811594


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
